In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision.datasets import STL10
from torchvision import transforms

In [2]:
transform=transforms.Compose([transforms.ToTensor()])
dataset=torchvision.datasets.MNIST(root='./data',download=True,train=True,transform=transform)

100%|██████████| 9912422/9912422 [00:01<00:00, 6705710.13it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 49870602.64it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 6055305.35it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4496230.53it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
print("dataset size",len(dataset))

dataset size 60000


In [4]:
totalsize=len(dataset)
train_size=int(0.6*totalsize)
valid_size=int(0.2*totalsize)
test_size=int(0.2*totalsize)
print("train size",train_size)
print("valid size",valid_size)
print("test size",test_size)

train size 36000
valid size 12000
test size 12000


In [5]:
train_dataset,valid_dataset,test_dataset=random_split(dataset,[train_size,valid_size,test_size] )

In [6]:
batch_size = 64
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [12]:
import torch.nn as nn
import torch.nn.functional as F

class MobileNet(nn.Module):
    def __init__(self): 
        super(MobileNet, self).__init__()  # Corrected here
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = MobileNet()


In [13]:
optimizer=optim.Adam(model.parameters(),lr=0.01)
criterion=nn.CrossEntropyLoss()

In [14]:
# training the model 
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10  


for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)

        
        optimizer.zero_grad()

       
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(trainloader)
    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

Epoch 1, Train Loss: 0.2072, Train Accuracy: 93.39%
Epoch 2, Train Loss: 0.0751, Train Accuracy: 97.67%
Epoch 3, Train Loss: 0.0670, Train Accuracy: 97.98%
Epoch 4, Train Loss: 0.0591, Train Accuracy: 98.22%
Epoch 5, Train Loss: 0.0544, Train Accuracy: 98.38%
Epoch 6, Train Loss: 0.0623, Train Accuracy: 98.17%
Epoch 7, Train Loss: 0.0535, Train Accuracy: 98.51%
Epoch 8, Train Loss: 0.0578, Train Accuracy: 98.38%
Epoch 9, Train Loss: 0.0485, Train Accuracy: 98.62%
Epoch 10, Train Loss: 0.0526, Train Accuracy: 98.56%


In [16]:
# Validation 
model.eval()  
running_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  
    for data in validloader:  
        images, labels = data[0].to(device), data[1].to(device) 
        outputs = model(images)  
        loss = criterion(outputs, labels)  

        running_loss += loss.item()  
        _, predicted = torch.max(outputs.data, 1) 
        total += labels.size(0)  
        correct += (predicted == labels).sum().item() 


val_loss = running_loss / len(validloader)
val_accuracy = 100 * correct / total


print(f"Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

print('Finished Training')


Epoch 10, Validation Loss: 0.0907, Validation Accuracy: 98.20%
Finished Training


In [17]:
# Testing
model.eval()  
running_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  
    for data in testloader:  
        images, labels = data[0].to(device), data[1].to(device) 
        outputs = model(images)  
        loss = criterion(outputs, labels) 

        running_loss += loss.item()  
        _, predicted = torch.max(outputs.data, 1)  
        total += labels.size(0) 
        correct += (predicted == labels).sum().item()  


test_loss = running_loss / len(testloader)
test_accuracy = 100 * correct / total


print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Test Loss: 0.1017, Test Accuracy: 97.94%
